In [51]:
import pandas as pd
import numpy as np
import sklearn
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.feature_selection import RFE, f_regression
from sklearn.linear_model import (LinearRegression, Ridge, Lasso, RandomizedLasso)
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import gaussian_kde
import sklearn.linear_model as lm
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

from sklearn.metrics import mean_squared_error as MSE
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn import cross_validation
from sklearn.grid_search import GridSearchCV

import seaborn as sns

In [63]:
# https://github.com/alexyalunin/econometrics/blob/master/data/kc_house_data_new.xls
data = pd.read_excel("kc_house_data.xls")
prices = data['price']
data = data.drop('price', axis=1)
data.head()

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,log_sqft_lot15,log_yr_renovated,bedrooms_bathrooms,bathrooms_log_living,view_log_condition,log_grade_exp_condition,log_condition,view_bedrooms,zipcode_lat,log_price
0,7129300520,20141013T000000,3,1.00,1180,5650,1.0,0,0,3,...,8.639411,7.608871,3.00,7.073270,0.0,39.084650,1.098612,0,4.664555e+06,12.309982
1,6414100192,20141209T000000,3,2.25,2570,7242,2.0,0,0,3,...,8.941022,3.218876,6.75,17.666238,0.0,39.084650,1.098612,0,4.682623e+06,13.195614
2,5631500400,20150225T000000,2,1.00,770,10000,1.0,0,0,3,...,8.994917,7.608871,2.00,6.646391,0.0,35.988451,1.098612,0,4.679651e+06,12.100712
3,2487200875,20141209T000000,4,3.00,1960,5000,1.0,0,0,5,...,8.517193,7.608871,12.00,22.742099,0.0,288.798673,1.609438,0,4.663501e+06,13.311329
4,1954400510,20150218T000000,3,2.00,1680,8080,1.0,0,0,3,...,8.923058,7.608871,6.00,14.853098,0.0,41.766700,1.098612,0,4.669970e+06,13.142166


In [35]:
data = data.iloc[:,2:]
# Looking for nulls
print(data.isnull().any())
# Inspecting type
print(data.dtypes)

bedrooms                   False
bathrooms                  False
sqft_living                False
sqft_lot                   False
floors                     False
waterfront                 False
view                       False
condition                  False
grade                      False
sqft_above                 False
sqft_basement              False
yr_built                   False
yr_renovated               False
zipcode                    False
lat                        False
long                       False
sqft_living15              False
sqft_lot15                 False
log_sqft_living            False
log_bedrooms               False
exp_bathrooms              False
log_lat                    False
log_zipcode                False
log_long                   False
log_view                   False
log_sqft_above             False
log_sqft_basement          False
log_sqft_lot15             False
log_yr_renovated           False
bedrooms_bathrooms         False
bathrooms_

In [36]:
data.columns

Index(['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15', 'log_sqft_living', 'log_bedrooms',
       'exp_bathrooms', 'log_lat', 'log_zipcode', 'log_long', 'log_view',
       'log_sqft_above', 'log_sqft_basement', 'log_sqft_lot15',
       'log_yr_renovated', 'bedrooms_bathrooms', 'bathrooms_log_living',
       'view_log_condition', 'log_grade_exp_condition', 'log_condition',
       'view_bedrooms', 'zipcode_lat', 'log_price'],
      dtype='object')

In [37]:
# with sns.plotting_context("notebook",font_scale=2.5):
#     g = sns.pairplot(house[['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
#        'waterfront', 'view', 'condition', 'grade', 'sqft_above',
#        'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
#        'sqft_living15', 'sqft_lot15']], 
#                  hue='floors', palette='tab20',size=6)
# g.set(xticklabels=[]);

In [ ]:
# with sns.plotting_context("notebook",font_scale=2.5):
#     g = sns.pairplot(house[['floors','bedrooms','bathrooms','grade','waterfront','view','condition']], 
#                  hue='grade', palette='tab20',size=6)
# g.set(xticklabels=[]);

In [61]:
# with sns.plotting_context("notebook",font_scale=2.5):
#     g = sns.pairplot(house[['sqft_lot','sqft_above','price','sqft_living','bedrooms']], 
#                  hue='bedrooms', palette='tab20',size=6)
# g.set(xticklabels=[]);
# with sns.plotting_context("notebook",font_scale=2.5):
#     g = sns.pairplot(data=house,
#                   y_vars=['price'],
#                   x_vars=['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot'], palette='tab20', size=6, kind='reg', plot_kws={'line_kws':{'color':'green'}, 'scatter_kws': {'alpha': 0.1}})
#     g1 = sns.pairplot(data=house,
#                   y_vars=['price'],
#                   x_vars=['floors', 'view', 'condition', 'grade'], palette='tab20', size=6, kind='reg', plot_kws={'line_kws':{'color':'green'}, 'scatter_kws': {'alpha': 0.1}})
#     g2 = sns.pairplot(data=house,
#                   y_vars=['price'],
#                   x_vars=['sqft_above','sqft_basement', 'yr_built', 'yr_renovated'], palette='tab20', size=6, kind='reg', plot_kws={'line_kws':{'color':'green'}, 'scatter_kws': {'alpha': 0.1}})
#     g3 = sns.pairplot(data=house,
#                   y_vars=['price'],
#                   x_vars=['sqft_living15', 'sqft_lot15'], palette='tab20', size=6, kind='reg', plot_kws={'line_kws':{'color':'green'}, 'scatter_kws': {'alpha': 0.1}})
# g.set(xticklabels=[]);
# g1.set(xticklabels=[]);
# g2.set(xticklabels=[]);
# g3.set(xticklabels=[]);
# with sns.plotting_context("notebook"):
#     g = sns.pairplot(data=house,
#                   y_vars=['price'],
#                   x_vars=['price'], palette='tab20', size=6, kind='reg', plot_kws={'line_kws':{'color':'green'}, 'scatter_kws': {'alpha': 0.1}})

# g.set(xticklabels=[]);
# markerSize = 1
# df = house[["price", "sqft_living"]]
# plt.scatter(df.price, df.sqft_living, c="darkgreen", marker="v", s=markerSize, alpha=.1)
# plt.title('Correlation between age and fare')
# plt.xlabel('Age of a passenger')
# plt.ylabel('Fare paid')
# plt.show()


# # Generate fake data
# x = data['log_sqft_living']
# y = data['log_price']



# xy = np.vstack([x,y])
# z = gaussian_kde(xy)(xy)

# fig, ax = plt.subplots()
# ax.scatter(x, y, c=z, s=10, edgecolor='')
# plt.xlabel('log_sqft_living')
# plt.ylabel('log_price')
# plt.show()

In [59]:
# with sns.plotting_context("notebook",font_scale=2.5):
#     g = sns.pairplot(data=house,
#                   y_vars=['price'],
#                   x_vars=['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot'], palette='tab20', size=6, kind='reg', plot_kws={'line_kws':{'color':'green'}, 'scatter_kws': {'alpha': 0.1}})
# g.set(xticklabels=[]);

In [60]:
# str_list = [] # empty list to contain columns with strings (words)
# for colname, colvalue in house.iteritems():
#     if type(colvalue[1]) == str:
#          str_list.append(colname)
# # Get to the numeric columns by inversion            
# num_list = house.columns.difference(str_list) 
# # Create Dataframe containing only numerical features
# house_num = house[num_list]
# f, ax = plt.subplots(figsize=(16, 12))
# plt.title('Pearson Correlation of features')
# # Draw the heatmap using seaborn
# #sns.heatmap(house_num.astype(float).corr(),linewidths=0.25,vmax=1.0, square=True, cmap="PuBuGn", linecolor='k', annot=True)
# sns.heatmap(house_num.astype(float).corr(),linewidths=0.25,vmax=1.0, square=True, cmap="PuBuGn", linecolor='k', annot=True)

In [40]:
data = data.fillna(data.mean())

data = data.reindex(np.random.permutation(data.index))
data_train, data_validate, data_test = np.array_split(data, 3)

X_train = data_train.iloc[:,:-1]
y_train = data_train.iloc[:,-1]
X_validate = data_validate.iloc[:,:-1]
y_validate = data_validate.iloc[:,-1]
X_test = data_test.iloc[:,:-1]
y_test = data_test.iloc[:,-1]

X_train_new = np.concatenate([X_train, X_validate], axis=0)
y_train_new = np.concatenate([y_train, y_validate], axis=0)

In [47]:
LR = lm.LinearRegression()
LR.fit(X_train_new, y_train_new)
y_pred = LR.predict(X_test)
e = MSE(y_test, y_pred)
e

0.061257995013975566

In [48]:
ridge = lm.Ridge()
gs = GridSearchCV(ridge, {'alpha': [0.0001, 0.01, 1, 10, 100, 200, 1000000]},)
gs.fit(X_validate, y_validate)

print("For Ridge: ")
print(gs.grid_scores_)
print(gs.best_params_)

las = lm.Lasso()
gs2 = GridSearchCV(las, {'alpha': [0.0001, 0.01, 1, 10, 100, 200, 1000000]},)
gs2.fit(X_validate, y_validate)

print("For Lasso: ")
print(gs2.grid_scores_)
print(gs2.best_params_)

For Ridge: 
[mean: 0.77662, std: 0.00090, params: {'alpha': 0.0001}, mean: 0.77660, std: 0.00093, params: {'alpha': 0.01}, mean: 0.77686, std: 0.00100, params: {'alpha': 1}, mean: 0.77669, std: 0.00105, params: {'alpha': 10}, mean: 0.77339, std: 0.00118, params: {'alpha': 100}, mean: 0.77170, std: 0.00113, params: {'alpha': 200}, mean: 0.69115, std: 0.00464, params: {'alpha': 1000000}]
{'alpha': 1}


/Users/Alexander/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/Alexander/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/Alexander/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/Alexander/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Obje

For Lasso: 
[mean: 0.77686, std: 0.00103, params: {'alpha': 0.0001}, mean: 0.76450, std: 0.00041, params: {'alpha': 0.01}, mean: 0.68655, std: 0.00217, params: {'alpha': 1}, mean: 0.67057, std: 0.00228, params: {'alpha': 10}, mean: 0.60730, std: 0.00504, params: {'alpha': 100}, mean: 0.47619, std: 0.00531, params: {'alpha': 200}, mean: -0.00043, std: 0.00031, params: {'alpha': 1000000}]
{'alpha': 0.0001}


/Users/Alexander/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [49]:
best_alpha_ridge = gs.best_params_['alpha']
best_alpha_lasso = gs2.best_params_['alpha']

ridge = lm.Ridge(alpha = best_alpha_ridge)
ridge.fit(X_train_new, y_train_new)
y_pred_ridge = ridge.predict(X_test)
err_ridge = MSE(y_pred_ridge, y_test)

lasso = lm.Lasso(alpha = best_alpha_lasso)
lasso.fit(X_train_new, y_train_new)
y_pred_lasso = lasso.predict(X_test)
err_lasso = MSE(y_pred_lasso, y_test)

print("For Ridge: ")
print(err_ridge)
print(ridge.coef_)

print("For Lasso: ")
print(err_lasso)
print(lasso.coef_)

For Ridge: 
0.061476487805
[  8.97003617e-03  -1.80485188e-01   1.48100241e-05   5.99646546e-07
   2.40047104e-02   4.47654455e-01   2.04809831e-02  -2.20031405e-03
   1.47145279e-01   2.26849138e-05  -7.87626898e-06  -3.39171275e-03
  -1.35547235e-04  -1.35907292e-03   8.45268465e-03  -1.21361146e-01
   1.09552569e-04   8.26628098e-07   1.18916506e-01  -9.84756040e-02
  -1.75111545e-04  -4.66937756e-26   2.33468878e-26   9.33875511e-26
   5.01740918e-02   1.45126813e-01   5.83314093e-03  -7.69672114e-02
  -6.86565450e-02  -9.61547698e-04   3.09644379e-02   2.59715985e-02
   1.54250154e-04   1.96775395e-01  -5.98963168e-03   1.39344610e-05]
For Lasso: 
0.0615239312183
[ -0.00000000e+00  -5.61288901e-03   2.18851017e-04   5.93875080e-07
   2.24596194e-02   4.33225678e-01   1.87024952e-02   5.92955573e-02
   1.48220584e-01  -1.49412320e-04  -1.83072667e-04  -3.42071719e-03
  -1.29133386e-04  -1.34862183e-03   0.00000000e+00  -1.12188014e-01
   1.08991418e-04   8.18274259e-07   1.01709910

/Users/Alexander/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [52]:
clf = sklearn.neighbors.KNeighborsRegressor(n_neighbors=10)
clf.fit(X_train_new, y_train_new)
y_pred = clf.predict(X_test)
e = MSE(y_test, y_pred)
e

0.073399244160856317

In [53]:
knr = KNeighborsRegressor(metric='minkowski')
n_neighbors = np.arange(1,30)
p = np.array((1,2))
params = {'n_neighbors' : n_neighbors, 'p' : p}

gscv = GridSearchCV(knr, params, scoring = 'neg_mean_squared_error')

gscv.fit(X_train_new, y_train_new)
gscv_best_n = gscv.best_params_['n_neighbors']
gscv_best_p = gscv.best_params_['p']
print(gscv_best_n)
print(gscv_best_p)

12
1


In [54]:
knr = KNeighborsRegressor(metric='minkowski', n_neighbors = gscv_best_n, p = gscv_best_p)
knr.fit(X_train_new, y_train_new)
y_pred = knr.predict(X_test)
err = MSE(y_pred, y_test)
err

0.063225291797438452